In [62]:
import dgl.data
import numpy as np
import scipy.sparse as sp

# Prepare training and testing sets

In [63]:
dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [64]:
u,v = g.edges()

In [54]:
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

In [55]:
train_pos_u

tensor([ 109, 1894, 1469,  ...,  106, 1169,  132])

In [76]:
# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
## negative edges: two nodes without an edge, ignore the same nodes
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
# All the negative edges for inference purpose
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [41]:
## Test coo_matrix
uu = np.array([2, 3, 6])
vv = np.array([3, 5, 4])
sm = sp.coo_matrix((np.ones(len(uu)), (uu, vv)), shape=(7,7))
# sm.todense()
adj_neg_test = 1 - sm.todense() - np.eye(7)

# Remove test edges during training
dgl.remove_edges works by creating a subgraph from the original graph, resulting in a copy and therefore could be slow for large graphs. If so, you could save the training and test graph to disk, as you would do for preprocessing.

In [57]:
train_g = dgl.remove_edges(g, eids[:test_size])

# GraphSage model
After the graph convolutional layers, the model then predicts the probability of existence of an edge by computing a score between the representations of both incident nodes with a function (e.g. an MLP or a dot product), which you will see in the next section.

In [59]:
from dgl.nn import SAGEConv
import torch.nn as nn

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [101]:
torch.randn(3, requires_grad=True)

tensor([ 1.3868,  1.3370, -1.9156], requires_grad=True)

In [102]:
torch.empty(3).random_(2)

tensor([1., 0., 0.])

# Apply edge
Compute a score for the edge existance based on the two node features

In [60]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            # Replace the node features with the trained embeddings of the real graph
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [61]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            # Replace the node features with the trained embeddings of the real graph
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']


# Positive graph & negative graph
DGL recommends you to treat the pairs of nodes as another graph, since you can describe a pair of nodes with an edge. In link prediction, you will have a positive graph consisting of all the positive examples as edges, and a negative graph consisting of all the negative examples. The positive graph and the negative graph will contain the same set of nodes as the original graph. This makes it easier to pass node features among multiple graphs for computation. As you will see later, you can directly feed the node representations computed on the entire graph to the positive and the negative graphs for computing pair-wise scores.

In [65]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [80]:
inf_neg_g = dgl.graph((neg_u, neg_v), num_nodes=g.number_of_nodes())

# Training
After you defined the node representation computation and the edge score computation, you can go ahead and define the overall model, loss function, and evaluation metric.

The loss function is simply binary cross entropy loss.

In [72]:
from sklearn.metrics import roc_auc_score
import torch
import itertools
import torch.nn.functional as F

hdim = 16
model = GraphSAGE(train_g.ndata['feat'].shape[1], hdim)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(hdim)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

def compute_auc_prob(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [75]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
# Chain model and pred param together for gradient decent
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.001)

# ----------- 4. training -------------------------------- #
all_logits = []
nepoches = 100
for e in range(nepoches):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    # Passing the embedding of the original graph to predictor
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
#     print("pos score shape: ", pos_score.shape)
#     print("neg score shape: ", neg_score.shape)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
with torch.no_grad():
    # Passing the embedding of the original graph to predictor
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))
    
# ----------- 6. Inference ------------------------ #
# predict scores for all the adj negative edges to get the scores for the remaining uknown edges
# inf_score = pred(adj_neg_g, h)

/home/chenmin/miniconda3/envs/graph/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 0.6849939823150635
In epoch 5, loss: 0.6808724403381348
In epoch 10, loss: 0.6757968664169312
In epoch 15, loss: 0.6696400046348572
In epoch 20, loss: 0.662234902381897
In epoch 25, loss: 0.6534261703491211
In epoch 30, loss: 0.6430772542953491
In epoch 35, loss: 0.6310964226722717
In epoch 40, loss: 0.6175148487091064
In epoch 45, loss: 0.6025368571281433
In epoch 50, loss: 0.5866270065307617
In epoch 55, loss: 0.5704719424247742
In epoch 60, loss: 0.5549805760383606
In epoch 65, loss: 0.5410902500152588
In epoch 70, loss: 0.5295112133026123
In epoch 75, loss: 0.5204719305038452
In epoch 80, loss: 0.5136623382568359
In epoch 85, loss: 0.5084903836250305
In epoch 90, loss: 0.504317045211792
In epoch 95, loss: 0.5006279945373535
AUC 0.8560957750275151


In [105]:
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    # Convert logits to probabilities
    pos_score = torch.sigmoid(pos_score)
    neg_score = torch.sigmoid(neg_score)
    print('AUC', compute_auc(pos_score, neg_score))

AUC 0.8560957750275151


In [107]:
neg_score

tensor([0.1333, 0.1893, 0.1445,  ..., 0.4991, 0.8361, 0.6793])

In [100]:
pos_score

tensor([1.5861, 0.6322, 1.9251,  ..., 1.9727, 0.3539, 1.8622])

In [97]:
pos_score = pred(test_pos_g, h)
neg_score = pred(test_neg_g, h)

In [86]:
pos_score

tensor([1.5861, 0.6322, 1.9251,  ..., 1.9727, 0.3539, 1.8622],
       grad_fn=<SelectBackward0>)

In [92]:
pos_score.shape

torch.Size([1055])

In [89]:
scores = torch.cat([pos_score, neg_score])
labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])

In [93]:
scores.shape

torch.Size([2110])

In [91]:
labels

tensor([1., 1., 1.,  ..., 0., 0., 0.])

In [108]:
with torch.no_grad():
    # Passing the embedding of the original graph to predictor
    inf_score = pred(inf_neg_g, h)
    inf_score = torch.sigmoid(inf_score)

In [109]:
inf_score

tensor([0.6120, 0.7635, 0.3922,  ..., 0.7044, 0.6340, 0.3476])